In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)
import numpy as np



In [ ]:
master_data_path = "/Users/sagardhal/Desktop/Practice/personal-stock/results/predictions_20250908_092155.parquet"   # example


if master_data_path.endswith(".csv"):
    df = pd.read_csv(predictions_path)
elif master_data_path.endswith(".parquet"):
    df = pd.read_parquet(predictions_path)
else:
    raise ValueError("Unsupported file format for predictions")


print("Mater Data loaded:", df.shape)

master_df = df.copy()
master_df.head()

In [ ]:
daily_preds_path = "/Users/sagardhal/Desktop/Practice/personal-stock/new_pred_logs/predictions_top2_asof_2025-09-12.csv"   # example


if daily_preds_path.endswith(".csv"):
    df = pd.read_csv(daily_preds_path)
elif daily_preds_path.endswith(".parquet"):
    df = pd.read_parquet(daily_preds_path)
else:
    raise ValueError("Unsupported file format for predictions")


print("Mater Data loaded:", df.shape)

daily_preds_df = df.copy()
daily_preds_df.head()

In [ ]:
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt

# --- 1) restrict baseline to training split ---
baseline = master_df.loc[master_df["split"] == "train", "rf_prob_30d"].dropna()
current  = daily_preds_df["rf_prob_30d"].dropna()

# --- 2) KS test ---
stat, pval = ks_2samp(baseline, current)
print(f"KS statistic: {stat:.4f}, p-value: {pval:.2e}")

# --- 3) visualize distributions ---
plt.hist(baseline, bins=50, alpha=0.5, label="Train", density=True)
plt.hist(current, bins=50, alpha=0.5, label="Production", density=True)
plt.legend()
plt.xlabel("rf_prob_30d")
plt.ylabel("Density")
plt.title("Prediction distribution drift (Train vs Production)")
plt.show()


In [ ]:
train_stats = master_df[master_df["split"]=="train"].describe().T
prod_stats  = daily_preds_df.describe().T


In [ ]:
train_stats.loc["rf_prob_30d"]

In [ ]:
prod_stats.loc["rf_prob_30d"]